In [2]:
import io
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import urllib.parse
import html5lib

In [4]:
# tournament level

tourn_id = 16917
tourn_id_str = "?tourn_id=" + str(tourn_id)
url_str = "https://www.tabroom.com/index/tourn/"

home_page = url_str + "index.mhtml" + tourn_id_str
entries = url_str + "fields.mhtml" + tourn_id_str
judges = url_str + "judges.mhtml" + tourn_id_str
pairings = url_str + "postings/index.mhtml" + tourn_id_str
results = url_str + "results/index.mhtml" + tourn_id_str
 
URL = home_page
resp = requests.get(URL)
 
if resp.status_code == 200:

    # # Using lxml
    # dom = etree.HTML(resp.text)
    # elements = dom.xpath("//span[@data-testid='TemperatureValue' and contains(@class,'CurrentConditions')]")
    # print(elements[0].text)
 
    # # Using BeautifulSoup
    soup = BeautifulSoup(resp.text, "lxml")
    # elements = soup.select('span[data-testid="TemperatureValue"][class^="CurrentConditions"]')
    # print(elements[0].text)
    #tables = pd.read_html(URL)
    #tables = pd.DataFrame(tables[0])
    #display(tables)
    #display(tables.iloc[2])




In [6]:
#parsing events from entry

def parse_events():
    req = Request(entries)
    html_page = urlopen(req)
    soup = BeautifulSoup(html_page, "lxml")
    
    event_list = soup.find_all('a', class_="blue full")

    event_dict = {}

    for entry in event_list:
        entry = str(entry)

        index = entry.find("event_id")
        clipped = entry[index + 9:]
        end_index = clipped.find('">')
        end_event_title = clipped.find('</a>')
        event_id = entry[(index + 9):(index + 9 + end_index)]
        event_title = clipped[end_index + 2:end_event_title].strip()
        event_dict[event_title] = event_id

    return event_dict

In [5]:
# getting participants
def get_participants():

    participant_dict = {}
    # maps event to the list of participants

    for event_name, event_id in parse_events().items():
        event_link = entries + "&event_id=" + event_id

        tables = pd.read_html(event_link)
        tables = pd.DataFrame(tables[0])
        participant_dict[event_name] = tables

In [26]:
# handling judges
def parse_judges():
    req = Request(judges)
    html_page = urlopen(req)
    soup = BeautifulSoup(html_page, "lxml")
    judge_events = soup.find_all('div', class_="odd nospace")
    paradigm_link_dict = {}
    judge_table = pd.DataFrame()
    table_list = []
    event_list = []
    paradigm_list = []
    judge_person_id_list = []
    for cur_string in judge_events:

        # parse event title
        cur_string = str(cur_string)
        cutoff1 = '"halfspacer"></span>'
        event_index = cur_string.find(cutoff1) + len(cutoff1)
        cutoff2 = '</span>'
        event_end_index = cur_string[event_index:].find(cutoff2)
        event_title = cur_string[event_index:event_index + event_end_index].strip()
        
        # parse paradigm urls
        list_url_index = cur_string.find('href=') + 6
        list_url_end_index = cur_string[list_url_index:].find('">')

        paradigm_url = "https://www.tabroom.com" + cur_string[list_url_index:list_url_index + list_url_end_index].strip()
        paradigm_link_dict[event_title] = paradigm_url
    
    # visit all and load
    for event_title, paradigm_url in paradigm_link_dict.items():
        req = Request(paradigm_url)
        html_page = urlopen(req)
        soup = BeautifulSoup(html_page, "lxml")
        tables = pd.read_html(paradigm_url)
        table = pd.DataFrame(tables[0])
        table_list.append(table)
 


        judge_entries = soup.find_all('tr')[1:]
        

        for entry_i in judge_entries:
            event_list.append(event_title)
            entry_i = str(entry_i)


            if entry_i.find('<a') != -1:        # if there is a link, find the paradigm
                startindex = entry_i.find('href=') + 6
                end_index = entry_i.find('" target="')
                judge_url = "https://www.tabroom.com" + entry_i[startindex:end_index]
                judge_person_id = int(judge_url[-4:])
                judge_person_id_list.append(judge_person_id)


                # visit url and get paradigm
                req = Request(judge_url)
                html_page = urlopen(req)
                soup = BeautifulSoup(html_page, "lxml")

                judge_paradigm = soup.find_all('div', class_="paradigm ltborderbottom")
                judge_paradigm = str(judge_paradigm)
                #removing html
                judge_paradigm = judge_paradigm.replace('<p>', '')
                judge_paradigm = judge_paradigm.replace('</p>', '')
                judge_paradigm = judge_paradigm.replace('[<div class="paradigm ltborderbottom">', '')
                judge_paradigm = judge_paradigm.replace('</div>]', '')
                judge_paradigm = judge_paradigm.strip()
                if len(judge_paradigm) > 0:
                    paradigm_list.append(judge_paradigm)
                else:
                    paradigm_list.append("No paradigm")

            else:
                judge_person_id_list.append(0)
                paradigm_list.append("No paradigm")


        # find way to parse entry for <a> tokens, extract url

        # parse judge person_id
        # parse judge paradigm - if neither exist add NA to a column in DF
    
    judge_table = pd.concat(table_list, ignore_index=True)
    judge_table = judge_table.drop(['Para','Rounds', 'Middle'], axis=1)
    judge_table['Event'] = event_list
    judge_table['Paradigm'] = paradigm_list
    judge_table['Judge ID'] = judge_person_id_list
    return judge_table

In [27]:
full_judge_list = parse_judges()


In [29]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(full_judge_list)

,First,Last,School,Location,Event,Paradigm,Judge ID
0,Terence,Abad,Lowell High School,CA,Con,"Congress judging paradigm\nNo canned speeches,...",8937
1,Ayyah,Al-jibouri,Mira Loma High School,CA,Con,<strong>TLDR: Former NFA-LD debater. Make the ...,3076
2,Sonal,Bahl,New Haven Academy,TX,Con,No paradigm,0
3,Bradley,Brown,Monte Vista,CA,Con,No paradigm,0
4,Sam,Campbell,Mountain House,CA,Con,I am a open debater and I will vote on almost ...,3667
5,Jenny,Darlington,1111111 Hire,CA,Con,"8-year Mock Trial Competitor, first time judgi...",613
6,Neha,Dubey,1111111 Hire,CA,Con,Debated PF for Mission San Jose for 4 years (a...,861
7,Susan,Foley,Campbell Hall High School,CA,Con,<strong>Parli</strong>\n- Don't generally like...,4311
8,Padma,Ganti,Northwood High School,CA,Con,I dont like spreading because I am a parent ju...,5190
9,Marc,Gittelman,Lowell High School,CA,Con,No paradigm,0
